**Exercício 10.2**

In [7]:
from si.io.csv_file import read_csv
from si.model_selection.split import train_test_split
from si.models.knn_classifier import KNNClassifier
from si.models.logistic_regression import LogisticRegression
from si.models.decision_tree_classifier import DecisionTreeClassifier
from si.ensemble.stacking_classifier import StackingClassifier
from si.metrics.accuracy import accuracy
from si.statistics.euclidean_distance import euclidean_distance


In [8]:
# 1) carregar dataset
dataset = read_csv("/Users/inesglameira/Documents/GitHub/SIB/datasets/breast_bin/breast-bin.csv", sep=",", features=True, label=True)



In [9]:
# 2) split treino/teste
train_ds, test_ds = train_test_split(dataset, test_size=0.2, random_state=42)

In [10]:
# 3) modelos base
knn = KNNClassifier(k=5, distance_function=euclidean_distance)
dt = DecisionTreeClassifier(max_depth=5)
logreg = LogisticRegression()

base_models = [knn, dt, logreg]

In [11]:
# 4) modelo final
final_model = KNNClassifier(k=3, distance_function=euclidean_distance)


In [12]:
# 5) stacking
stacking = StackingClassifier(
    models=base_models,
    final_model=final_model
)

In [13]:
# 6) treino
stacking.fit(train_ds)

In [14]:
# 7) avaliação
y_pred = stacking.predict(test_ds)
score = accuracy(test_ds.y, y_pred)

print("Accuracy do StackingClassifier:", score)

Accuracy do StackingClassifier: 0.9803220035778175


O desempenho do StackingClassifier foi avaliado no conjunto de teste utilizando a métrica accuracy.
O modelo obteve uma accuracy de cerca de 98%, demonstrando que a abordagem de stacking foi eficaz, ao combinar as previsões dos modelos base para melhorar a performance global do classificador.

**Exercício 11.2**

In [19]:
import numpy as np

from si.io.csv_file import read_csv
from si.model_selection.randomized_search import randomized_search_cv
from si.models.knn_classifier import KNNClassifier
from si.metrics.accuracy import accuracy
from si.statistics.euclidean_distance import euclidean_distance


In [17]:
#1 Carregar dataset
dataset = read_csv("/Users/inesglameira/Documents/GitHub/SIB/datasets/breast_bin/breast-bin.csv", sep=",", features=True, label=True)

In [20]:
#2 Carregar modelo
model = KNNClassifier(
    k=5,
    distance_function=euclidean_distance
)


In [21]:
#3 Definir o espaço de hiperparâmetros
param_grid = {
    "k": np.array([1, 3, 5, 7, 9]),
    "distance_function": np.array([euclidean_distance])
}

In [22]:
#4 Executar Randomized Search 
results = randomized_search_cv(
    model=model,
    dataset=dataset,
    hyperparameter_grid=param_grid,
    scoring=accuracy,
    cv=5,
    n_iter=10,
    random_state=42
)

In [23]:
#5 Mostrar os resultados
print("Melhores hiperparâmetros:")
print(results["best_hyperparameters"])

print("\nMelhor score obtido:")
print(results["best_score"])

Melhores hiperparâmetros:
{'k': np.int64(7), 'distance_function': <function euclidean_distance at 0x10ca16340>}

Melhor score obtido:
0.9697841726618706


O Randomized Search permitiu identificar a configuração ótima do KNN, com k = 7 e distância Euclidiana, alcançando uma accuracy média de cerca de 97%, superior às restantes combinações testadas.

**Exercicio 12.2**

In [25]:
import numpy as np
from si.neural_networks.layers import Dropout

In [26]:
# Fixar seed para reprodutibilidade
np.random.seed(42)

In [27]:
# Criar input aleatório
X = np.random.rand(4, 5)  # batch_size=4, features=5

In [33]:
# Criar camada Dropout com p=0.5
dropout = Dropout(p=0.5)
print("Input original:")
print(X)

Input original:
[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]
 [0.15599452 0.05808361 0.86617615 0.60111501 0.70807258]
 [0.02058449 0.96990985 0.83244264 0.21233911 0.18182497]
 [0.18340451 0.30424224 0.52475643 0.43194502 0.29122914]]


In [34]:
# Forward propagation (treino)

out_train = dropout.forward_propagation(X, training=True)

print("\nOutput em modo treino (Dropout ativo):")
print(out_train)


Output em modo treino (Dropout ativo):
[[0.         0.         0.         1.19731697 0.        ]
 [0.31198904 0.         1.73235229 1.20223002 0.        ]
 [0.04116899 1.9398197  1.66488528 0.42467822 0.36364993]
 [0.36680902 0.         0.         0.         0.        ]]


In [35]:
# Forward propagation (inferência)
out_test = dropout.forward_propagation(X, training=False)

print("\nOutput em modo inferência (sem Dropout):")
print(out_test)


Output em modo inferência (sem Dropout):
[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864]
 [0.15599452 0.05808361 0.86617615 0.60111501 0.70807258]
 [0.02058449 0.96990985 0.83244264 0.21233911 0.18182497]
 [0.18340451 0.30424224 0.52475643 0.43194502 0.29122914]]


In [36]:
# Backward propagation
output_error = np.ones_like(X)
back_error = dropout.backward_propagation(output_error)

print("\nErro propagado no backward:")
print(back_error)


Erro propagado no backward:
[[0. 0. 0. 2. 0.]
 [2. 0. 2. 2. 0.]
 [2. 2. 2. 2. 2.]
 [2. 0. 0. 0. 0.]]


In [37]:
# Verificações adicionais
print("\nForma do input:", X.shape)
print("Forma do output:", out_train.shape)
print("Número de parâmetros:", dropout.parameters())


Forma do input: (4, 5)
Forma do output: (4, 5)
Número de parâmetros: {}


A camada Dropout foi testada com um input aleatório de dimensão (4,5). Verificou-se que a forma do output se mantém igual à do input, confirmando que a camada não altera a dimensionalidade dos dados.

O método parameters() devolveu um dicionário vazio, indicando que a camada Dropout não possui parâmetros treináveis. Durante o treino, a camada desliga aleatoriamente alguns neurónios através de uma máscara binária, enquanto em modo de inferência os dados não são alterados.

Estes resultados confirmam o comportamento esperado da camada Dropout.